In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import random
import math
import time 
from scipy.optimize import minimize
from scipy.stats import norm
import ray
ray.init(num_cpus= 10, ignore_reinit_error=True)
from numba import jit

2023-02-21 23:00:49,163	INFO worker.py:1538 -- Started a local Ray instance.


In [59]:
#define true parameters 
alpha_1 = 1
alpha_2 = 1
beta_loc = -0.1
beta_scale = 1
price_loc = 2
price_scale = 0.5

# Part A. Simulate Data

In [48]:
def DGP():
    # create frame of the dataset 
    # i X t dataframe shape: (1000 X 2)
    i_list = np.full(20, 1)
    t_list = np.array([t for t in range(1, 21)])
    for i in range(2,101):
        i_list = np.concatenate((i_list, np.full(20, i)), axis = None)
        t_list = np.concatenate((t_list, np.array([t for t in range(1, 21)])), axis = None)
    df_simulation = df({'i': i_list, 't': t_list})

    # t p_1t p_2t dataframe (10 X 2)
    # price is varying across time 
    t_list_20 = np.array([t for t in range(1, 21)])
    p_1_draw = np.random.normal(price_loc, price_scale, 20)
    p_2_draw = np.random.normal(price_loc, price_scale, 20)
    df_price = df({'t': t_list_20, 'p_1t': p_1_draw, 'p_2t': p_2_draw})

    
    # draw beta for each i 
    # and merge it to the df_simulation
    i_list_100 = np.array([i for i in range(1, 101)])
    beta_draw = np.random.normal(beta_loc, beta_scale, 100)
    df_beta = df({"i": i_list_100, "beta": beta_draw})

    # merge
    df_simulation = pd.merge(df_simulation, df_price, left_on="t", right_on="t")
    df_simulation = pd.merge(df_simulation, df_beta, left_on="i", right_on="i")
    df_simulation = df_simulation.sort_values(by = ['i', 't'], ascending= True)

    # simulate decision
    p_1_array = np.array(df_simulation['p_1t'])
    p_2_array = np.array(df_simulation['p_2t'])
    beta_array = np.array(df_simulation['beta'])
    exp_1_array = np.exp(alpha_1 + beta_array*p_1_array)
    exp_2_array = np.exp(alpha_2 + beta_array*p_2_array)
    prob_1_array = exp_1_array/(1 + exp_1_array + exp_2_array)
    prob_2_array = exp_2_array/(1 + exp_1_array + exp_2_array)
    prob_0_array = 1 - prob_1_array - prob_2_array
    u_array = np.random.uniform(0,1,2000)

    # let the error term (epsilon) determine the decision
    # outside option <-- left extreme
    d_0t_bool = (u_array < prob_0_array).astype(int)
    # decision for product 1 <-- middle part
    d_1t_temp1 = (u_array >= prob_0_array)
    d_1t_temp2 = (u_array < prob_0_array + prob_1_array)
    d_1t_bool = (d_1t_temp1 * d_1t_temp2).astype(int)
    # decision for product 2 <-- right extreme
    d_2t_bool = (u_array >= prob_0_array + prob_1_array).astype(int)

    df_simulation['d_0t'] = d_0t_bool
    df_simulation['d_1t'] = d_1t_bool
    df_simulation['d_2t'] = d_2t_bool

    return df_simulation


In [60]:
#data_sample = DGP()
#data_sample.to_csv("data_sample.csv")
data_sample= pd.read_csv("data_sample.csv")

# Part B. Simulated MLE

In [61]:
@jit
def multiplyAll(s):
    ans = 1
    for n in s:
        ans *= n
    return ans

In [62]:
def MLE(theta, data_sample):
    alpha_1, alpha_2, beta_loc, beta_scale = theta
    
    # parallelize for loop on individual i
    @ray.remote
    def i_prob(i):
        work_data = data_sample.loc[data_sample['i']==i]
        np.random.seed(i)
        beta_draw = np.random.uniform(-2,2,300) # number of draw matters for the MLE estimates
                                                # higher number of beta draw will make the integration precise
                                                # Question: How do we set up lower and upper limit of beta? 

        # within each i, calculate the likelihood
        def prob_calculate(beta):
            exp_1_array = np.exp(alpha_1 + beta*work_data['p_1t'].values)
            exp_2_array = np.exp(alpha_2 + beta*work_data['p_2t'].values)
            prob_1_array = exp_1_array/(1 + exp_1_array + exp_2_array)
            prob_2_array = exp_2_array/(1 + exp_1_array + exp_2_array)
            prob_0_array = 1 - prob_1_array - prob_2_array
            prob_array = prob_0_array*work_data['d_0t'].values + prob_1_array*work_data['d_1t'].values + prob_2_array*work_data['d_2t'].values
            conditional_prob = multiplyAll(prob_array) # probability condiotinal on beta
            prob = conditional_prob * norm.pdf(beta, beta_loc, beta_scale) # unconditional probability
            return prob

        vec_prob_calculate = np.vectorize(prob_calculate)
        # take average of the probability for 300 many drawn beta
        mc_prob = vec_prob_calculate(beta_draw).mean()
        # return the log likelihood
        return -np.log(mc_prob)

    likelihood_list = [i_prob.remote(i) for i in range(1,101)]
    likelihood_array = np.array(ray.get(likelihood_list))

    return likelihood_array.sum()

 

In [63]:
MLE_result = minimize(MLE, [0.5, 0.5, 0, 0.5], args = (data_sample), method='L-BFGS-B', options={'maxiter':200})
MLE_result.x

array([ 1.09464869,  1.05299484, -0.07894199,  0.85991289])

# Part C. Hierarchical Bayes 

* Here I assume that a researcher aleready knows $\alpha_1$ and $\alpha_2$

In [11]:
def personal_llh(beta_candidate, personal_data):

    p_1_array = np.array(personal_data['p_1t'])
    p_2_array = np.array(personal_data['p_2t'])
    d_0t_array = np.array(personal_data['d_0t'])
    d_1t_array = np.array(personal_data['d_1t'])
    d_2t_array = np.array(personal_data['d_2t'])
    
    exp_1_array = np.exp(alpha_1 + beta_candidate*p_1_array)
    exp_2_array = np.exp(alpha_2 + beta_candidate*p_2_array)
    prob_1_array = exp_1_array/(1 + exp_1_array + exp_2_array)
    prob_2_array = exp_2_array/(1 + exp_1_array + exp_2_array)
    prob_0_array = 1 - prob_1_array - prob_2_array

    prob_array = d_0t_array * np.log(prob_0_array) + d_1t_array * np.log(prob_1_array) + d_2t_array * np.log(prob_2_array)
    log_sum = prob_array.sum()

    return log_sum    

In [36]:
def inverted_gamma_draw(s_bar):
    nu_1 = 101 
    s_1 = (1 + 100*s_bar)/101

    eta_draw = np.random.normal(0,1,nu_1)
    eta_temp = (eta_draw/np.sqrt(s_1))**2
    r = eta_temp.mean()

    return 1/r

In [53]:
beta_dump = []
mu_dump = []
sigma_dump = []

mu_pre = 0
sigma_pre = 0.5
beta_pre = np.zeros(100)

for t in range(2000):
    #1. update beta
    beta_post = []
    for i in range(1,101):
        # draw new beta
        personal_data = data_sample.loc[data_sample['i']==i]
        personal_beta_candidate = np.random.normal(mu_pre, sigma_pre, 1)[0]
        personal_beta_pre = beta_pre[i-1]

        # decide accept/reject
        criteria_mu = np.random.uniform(0,1,1)
        llh_pre = personal_llh(personal_beta_pre, personal_data)
        llh_candidate = personal_llh(personal_beta_candidate, personal_data)

        # update 
        if math.log(criteria_mu) <= llh_candidate - llh_pre:
            beta_post.append(personal_beta_candidate)
        else:
            beta_post.append(personal_beta_pre)
    
    beta_dump.append(np.array(beta_post))

    #2. update mu
    mu_loc = np.array(beta_post).mean()
    mu_scale = sigma_pre/10
    mu_post = np.random.normal(mu_loc, mu_scale, 1)[0]
    mu_dump.append(mu_post)

    #3. update sigma
    #calculate s_bar
    beta_bar = np.array(beta_post).mean()
    error_array = np.array(beta_post) - beta_bar
    sum_sq_error = error_array**2
    s_bar = sum_sq_error.mean()
    sigma_post = inverted_gamma_draw(s_bar)
    sigma_dump.append(sigma_post)

    #4. iterate
    beta_pre = beta_post
    mu_pre = mu_post
    sigma_pre = sigma_post


In [67]:
print("Bayesian estimates for beta_loc: ", np.array(beta_dump[500:2000]).mean())
print("Bayesian estimates for beta_scale: ", np.array(sigma_dump[500:2000]).mean())

Bayesian estimates for beta_loc:  -0.46974504683347146
Bayesian estimates for beta_scale:  0.06287205266837353


In [66]:
print("Bayesian estimates for beta_loc: ", np.array(beta_dump[:]).mean())
print("Bayesian estimates for beta_scale: ", np.array(sigma_dump[:]).mean())

Bayesian estimates for beta_loc:  -0.4706515373711152
Bayesian estimates for beta_scale:  0.06613425922290624
